In [99]:
import Data_Loader
from importlib import reload
reload(Data_Loader)
import os
import random

folder = "/mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks"

files = [entry.path for entry in os.scandir(folder) if entry.is_file()]

if len(files) > 10000:
    files = random.sample(files, 10000)

data_graphs = []
for f in files:
    try:
        d = Data_Loader.load_file(f)
        Data_Loader.aggregate_sequences(d)
        data_graphs.append(d)
    except Exception as e:
        print(f"Fehler beim Laden von {f}: {e}")

data = []
for G in data_graphs:
    try:
        dat = Data_Loader.graph_to_dataset(G)
        data.append(dat)
    except Exception as e:
        print(f"Fehler beim Laden von {G}: {e}")


"""
for node, attrs in data_graphs[0].nodes(data=True):
    seq = attrs.get("sequences", None)
    if seq is not None:
        print(node, seq[:50])   # nur die ersten 50 Stellen
"""

data_sample = random.choice(data)
#example = load_file(file)


Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1062.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1084.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1135.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1209.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1213.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation_1408.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks/simulation

In [100]:
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
import numpy as np
reload(Data_Loader)

class ParentChildFusionLayer(MessagePassing):
    """
    A MessagePassing layer designed for tree-like graphs where each parent
    has either exactly two children or none. For each parent node i, the
    features of the parent and its two children (if present) are concatenated.

    This layer does not use any attention or permutation-invariant
    aggregation: child messages are collected explicitly and concatenated
    in a fixed order. The user must ensure that each parent node has either
    (0 or 2) incoming edges, and that the edge_index ordering encodes a
    consistent left/right child order.

    Input dimensions:
        - Node feature dimension: in_dim
        - Output feature dimension: out_dim

    Output:
        - New node embeddings of dimension out_dim
    """

    def __init__(self, in_dim):
        # We do not use built-in aggregation ("add", "mean", ...) because
        # we aggregate manually. Set aggr=None.
            
        # Each node will produce: [parent_features, child1, child2]
        # If a node has no children, child features are zero-padded.
        super().__init__(node_dim=0, aggr=None)

        self.in_dim = in_dim

    def message(self, x_j):
        return x_j

    def aggregate(self, inputs, index, ptr=None, dim_size=None):
        """
        Collect exactly two child feature vectors per parent.

        inputs:  (num_edges, in_dim)
        index:   (num_edges,) target node for each edge

        Returns:
            A tensor of shape (num_nodes, 2 * in_dim) containing the two
            children features for each parent. Order is determined by
            edge ordering and should be consistent in the dataset.
        """

        # Determine number of nodes from dim_size (preferred), fall back to index
        if dim_size is not None:
            num_nodes = dim_size
        else:
            num_nodes = int(index.max().item()) + 1

        device = inputs.device

        # Preallocate storage
        children = torch.zeros(num_nodes, 2, self.in_dim, device=device)

        # Compute for each edge its "child slot" 0 or 1
        # Example: for index = [3,3,5,5], this produces [0,1,0,1]
        slot = torch.zeros_like(index)
        slot[1:] = (index[1:] == index[:-1]).long()

        # Vectorized scatter operation
        children[index, slot] = inputs

        return children.reshape(num_nodes, 2 * self.in_dim)

    def update(self, aggr_out, x):
        """
        aggr_out: (num_nodes, 2*in_dim) concatenated children features
        x:        (num_nodes, in_dim)   parent features

        Returns:
            Fused parent representation → out_dim
        """
        fused = torch.cat([x, aggr_out], dim=-1)
        return fused

    def forward(self, x, edge_index):
        """
        x: (num_nodes, in_dim)
        edge_index: (2, num_edges), where edges point child -> parent

        Returns:
            Updated node embeddings (num_nodes, out_dim)
        """
        return self.propagate(edge_index, x=x)


class RecipientFinder(nn.Module):
    """
    A model that stacks several ParentChildFusionLayers followed by
    fully connected layers for prediction.

    Parameters
    ----------
    in_channels : int
        Dimensionality of the input node features.
    hidden_channels : int
        Dimensionality after the ParentChildFusionLayers.
    num_fusion_layers : int
        Number of ParentChildFusionLayers to apply sequentially.
    dropout : float
        Dropout probability for the fully connected classifier.
    """

    def __init__(self, in_channels, hidden_channels = 16, num_fusion_layers=1, num_fc_layers = 2, dropout=0.3):

        super().__init__()

        # ----- Fusion Layers -----
        self.fusion_layers = nn.ModuleList()
        current_dim = in_channels

        for _ in range(num_fusion_layers):
            self.fusion_layers.append(ParentChildFusionLayer(in_dim=current_dim))
            current_dim = current_dim * 3 # 3 is coming from one parent and two children. If there are no children it is filled with zeros.

        # ----- Fully Connected Layers -----
        self.fc_layers = nn.ModuleList()
        self.dropout = dropout

        # First FC layer: (current_dim → hidden_channels) OR directly → 1
        if num_fc_layers == 1:
            self.fc_layers.append(nn.Linear(current_dim, 1))
        else:
            # First hidden layer
            self.fc_layers.append(nn.Linear(current_dim, hidden_channels))
            # Middle hidden FC layers
            for _ in range(num_fc_layers - 2):
                self.fc_layers.append(nn.Linear(hidden_channels, hidden_channels))
            # Final output layer
            self.fc_layers.append(nn.Linear(hidden_channels, 1))

    def forward(self, x, edge_index):
        """
        Apply stacked fusion layers followed by linear classifiers.
        """

        # ----- ParentChildFusionLayers -----
        for layer in self.fusion_layers:
            x = layer(x, edge_index)

        # ----- Fully Connected Layers -----
        for i, layer in enumerate(self.fc_layers):
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x)
            if i < len(self.fc_layers) - 1:
                x = F.relu(x)

        return x.view(-1)


i = 1
model = RecipientFinder(in_channels = data[i].x.shape[1])
#print(model)
model(data[i].x, data[i].edge_index)

tensor([     8.1,     -0.8,     -8.6,     19.2,     -5.3,      8.1,     -0.1,
            19.2,      8.4,     -5.6,      2.1,      6.2,     17.0,     13.4,
             1.3,      0.4,     -0.1,     16.7,     15.0,     -0.7,     -0.2,
            19.4,     10.1,     12.6,     13.4,     -4.0,     -0.1,      0.9,
             7.8,     -3.3,     -2.3,     13.3,      7.7,     -0.3,      4.8,
            -0.2,      6.2,     -2.3,      4.9,     10.3,      9.7,     -9.0,
            16.3,     12.7,     -0.1,     -2.5,      0.0,      8.8,     10.5,
             6.0,      4.0,     -0.2,     -0.3,     -0.2,     -0.2,      0.0,
            -0.0,     14.7,      7.5,      9.6,      1.4,     -2.8,     -2.6,
             4.0,      3.9,     -0.1,     -0.1,     -0.1,     -0.2,      0.1,
            -0.1,      8.0,     -0.2,     -2.0,     19.3,     -0.2,     -0.2,
            -0.1,      0.1,     19.7,     13.4,      6.6,     13.4,     -0.2,
            -0.2,     -0.2,      7.8,     10.6,      8.0,     14

In [ ]:
import Data_Loader
reload(Data_Loader)
from torch_geometric.loader import DataLoader


# === 1. Modell, Optimizer, Loss ===

model = RecipientFinder(in_channels=data[0].x.shape[1])

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in data])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 2. Training & Evaluation ===

# Train/Test Split
random.shuffle(data)
split_idx = int(0.9 * len(data))
train_data = data[:split_idx]
test_data = data[split_idx:]

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8)

def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader, threshold=0.5):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > threshold
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

@torch.no_grad()
def find_best_threshold(loader, thresholds=np.linspace(0, 1, 101)):
    model.eval()
    best_threshold = 0.5
    best_f1 = 0.0

    # Alle Outputs und Labels sammeln, damit man nicht für jeden Threshold neu durch die Daten geht
    all_outs = []
    all_labels = []
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        all_outs.append(torch.sigmoid(out))
        all_labels.append(batch.y)
    all_outs = torch.cat(all_outs)
    all_labels = torch.cat(all_labels)

    for threshold in thresholds:
        preds = all_outs > threshold
        tp = ((preds == 1) & (all_labels == 1)).sum().item()
        fp = ((preds == 1) & (all_labels == 0)).sum().item()
        fn = ((preds == 0) & (all_labels == 1)).sum().item()

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

# === 3. Training starten ===
for epoch in range(1, 21):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

# Nach Training besten Threshold bestimmen
best_threshold, best_f1 = find_best_threshold(test_loader)
print(f"\nBester Threshold: {best_threshold:.3f} mit F1-Score: {best_f1:.3f}")

# Evaluation mit bestem Threshold
acc, prec, rec, f1 = evaluate(test_loader, threshold=best_threshold)
print(f"Evaluation mit bestem Threshold:")
print(f"Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

/tmp/ipykernel_86/262190323.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Pos Weight: 10.64
Epoch 01 | Loss: 0.1995 | Acc: 0.980 | Prec: 0.235 | Rec: 0.529 | F1: 0.326
Epoch 02 | Loss: 0.1414 | Acc: 0.980 | Prec: 0.249 | Rec: 0.593 | F1: 0.351
Epoch 03 | Loss: 0.1332 | Acc: 0.984 | Prec: 0.307 | Rec: 0.543 | F1: 0.393
Epoch 04 | Loss: 0.1271 | Acc: 0.989 | Prec: 0.378 | Rec: 0.327 | F1: 0.351
Epoch 05 | Loss: 0.1217 | Acc: 0.982 | Prec: 0.294 | Rec: 0.644 | F1: 0.403
Epoch 06 | Loss: 0.1168 | Acc: 0.982 | Prec: 0.302 | Rec: 0.713 | F1: 0.424
Epoch 07 | Loss: 0.1101 | Acc: 0.983 | Prec: 0.324 | Rec: 0.768 | F1: 0.456
Epoch 08 | Loss: 0.1058 | Acc: 0.983 | Prec: 0.332 | Rec: 0.776 | F1: 0.465
Epoch 09 | Loss: 0.1040 | Acc: 0.989 | Prec: 0.436 | Rec: 0.570 | F1: 0.494
Epoch 10 | Loss: 0.1030 | Acc: 0.988 | Prec: 0.403 | Rec: 0.713 | F1: 0.515
Epoch 11 | Loss: 0.1021 | Acc: 0.989 | Prec: 0.439 | Rec: 0.636 | F1: 0.519
Epoch 12 | Loss: 0.1012 | Acc: 0.990 | Prec: 0.482 | Rec: 0.506 | F1: 0.494
Epoch 13 | Loss: 0.0969 | Acc: 0.989 | Prec: 0.431 | Rec: 0.668 | F1: 

In [98]:
class DonorFinder(nn.Module):
    """
    Scores candidate donors relative to a recipient vector.
    Returns raw scores (logits) of shape [M]
    """
    def __init__(self, in_channels, hidden_channels = 16, num_fusion_layers=3, num_fc_layers = 2, dropout=0.3):
        super().__init__()
        
        self.fc_layers = nn.ModuleList()
        self.dropout = dropout

        # First FC layer: (current_dim → hidden_channels) OR directly → 1
        if num_fc_layers == 1:
            self.fc_layers.append(nn.Linear(current_dim, 1))
        else:
            # First hidden layer
            self.fc_layers.append(nn.Linear(current_dim, hidden_channels))
            # Middle hidden FC layers
            for _ in range(num_fc_layers - 2):
                self.fc_layers.append(nn.Linear(hidden_channels, hidden_channels))
            # Final output layer
            self.fc_layers.append(nn.Linear(hidden_channels, 1))

    def forward(self, recipient_vec, donor_vecs):
                
        # ----- Fully Connected Layers -----
        for i, layer in enumerate(self.fc_layers):
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x)
            if i < len(self.fc_layers) - 1:
                x = F.relu(x)

        return x.view(-1)



In [ ]:
import Data_Loader
reload(Data_Loader)
from torch_geometric.loader import DataLoader


# === 1. Modell, Optimizer, Loss ===

model = RecipientFinder(in_channels=data[0].x.shape[1])

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in data])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 2. Training & Evaluation ===

# Train/Test Split
random.shuffle(data)
split_idx = int(0.5 * len(data))
train_data = data[:split_idx]
test_data = data[split_idx:]

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(train_data, batch_size=8)

def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader, threshold=0.5):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > threshold
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

@torch.no_grad()
def find_best_threshold(loader, thresholds=np.linspace(0, 1, 101)):
    model.eval()
    best_threshold = 0.5
    best_f1 = 0.0

    # Alle Outputs und Labels sammeln, damit man nicht für jeden Threshold neu durch die Daten geht
    all_outs = []
    all_labels = []
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        all_outs.append(torch.sigmoid(out))
        all_labels.append(batch.y)
    all_outs = torch.cat(all_outs)
    all_labels = torch.cat(all_labels)

    for threshold in thresholds:
        preds = all_outs > threshold
        tp = ((preds == 1) & (all_labels == 1)).sum().item()
        fp = ((preds == 1) & (all_labels == 0)).sum().item()
        fn = ((preds == 0) & (all_labels == 1)).sum().item()

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

# === 3. Training starten ===
for epoch in range(1, 21):
    loss = train()
    acc, prec, rec, f1 = evaluate(train_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

# Nach Training besten Threshold bestimmen
best_threshold, best_f1 = find_best_threshold(test_loader)
print(f"\nBester Threshold: {best_threshold:.3f} mit F1-Score: {best_f1:.3f}")

# Evaluation mit bestem Threshold
acc, prec, rec, f1 = evaluate(test_loader, threshold=best_threshold)
print(f"Evaluation mit bestem Threshold:")
print(f"Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")